In [ ]:
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
            

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
        
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [ ]:
#coding=utf-8
import xlwings as xw
import re
# from modules.my_libs import han_ji_chu_im_python as ji
from nbpackage import han_ji_chu_im as ji

#==========================================================
# Excel 檔案
#==========================================================
file_path = 'hoo-goa-chu-im.xlsx'
wb = xw.Book(file_path)

source_sheet = wb.sheets['漢字注音表']
end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

In [ ]:
#==========================================================
# 設定輸出使用的注音方法
#==========================================================
CONFIG_IDX = 3

zhu_im_config = {
    'sip_ngoo_im': [
      'fifteen',    # <div class="">
      'rt',         # Ruby Tag: <rt> / <rtc>  
      '十五音注音',   # 輸出工作表名稱
    ],
    'TPS': [
      'zhu_yin',    # <div class="">
      'rtc',        # Ruby Tag: <rt> / <rtc>  
      '方音符號注音', # 輸出工作表名稱
    ],
    'POJ': [
      'zhu_yin',    # <div class="">
      'rt',         # Ruby Tag: <rt> / <rtc>  
      '白話字拼音',   # 輸出工作表名稱
    ],
    'TL': [
      'zhu_yin',    # <div class="">
      'rt',         # Ruby Tag: <rt> / <rtc>  
      '台羅拼音',    # 輸出工作表名稱
    ],
    'BP': [
      'zhu_yin',    # <div class="">
      'rt',         # Ruby Tag: <rt> / <rtc>  
      '閩拼拼音',    # 輸出工作表名稱
    ],
}

zhu_im_method = list(zhu_im_config)[CONFIG_IDX]
div_class = zhu_im_config[zhu_im_method][0]
rt_tag = zhu_im_config[zhu_im_method][1]
zhu_im_sheet_name = zhu_im_config[zhu_im_method][2]

#==========================================================
# 備妥程式需使用之工作表
#==========================================================
sheet_name_list = ["無注音表"]
sheet_name_list.append(zhu_im_sheet_name)

#-----------------------------------------------------
# 檢查工作表是否已存在
for sheet_name in sheet_name_list:
    sheet = wb.sheets[sheet_name]
    try:
        sheet.select()
        continue
    except: 
        # CommandError 的 Exception 發生日，表工作表不存在
        # 新增程式需使用之工作表
        wb.sheets.add(name=sheet_name)

chu_im_sheet = wb.sheets[ zhu_im_sheet_name ]

In [ ]:
#=========================================================
# 輸出 <div> tag 
#=========================================================
i = 1       # index for target sheet
html_str = f"<div class='{div_class}'><p>"
chu_im_sheet.range('A' + str(i)).value = html_str
i += 1

row = 1     # index for source sheet
end_counter = end_row + 1

while row < end_counter:
    # 自 source_sheet 取待注音漢字
    han_ji = str(source_sheet.range('A' + str(row)).value)
    han_ji.strip()
    
    #=========================================================
    # 如是空白或換行，處理換行
    #=========================================================
    if han_ji == '' or han_ji == '\n':
        html_str = "</p><p>"
        chu_im_sheet.range('A' + str(i)).value = html_str
        i += 1
        row += 1
        continue
    
    #=========================================================
    # 如只是標點符號，不必處理為漢字注音的工作
    #=========================================================
    piau_tiam = r"[；：？！\uFF0C\uFF08-\uFF09\u2013-\u2014\u2026\\u2018-\u201D\u3000\u3001-\u303F]"
    searchObj = re.search(piau_tiam, han_ji, re.M|re.I)
    if searchObj:
        chu_im_sheet.range('A' + str(i)).value = han_ji
        i += 1
        row += 1
        continue
        
    #=========================================================
    # 在字庫中查不到注音的漢字，略過注音處理
    #=========================================================
    chu_im_code = str(source_sheet.range('B' + str(row)).value).strip()
    if chu_im_code == 'None':
        chu_im = ''
        ruby_tag = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><{rt_tag}>{chu_im}</{rt_tag}><rp>)</rp></ruby>'
        # if zhu_im_method == 'sip_ngoo_im':
        #     ruby_tag = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><rtc>{chu_im}</rtc><rp>)</rp></ruby>'
        # else:
        #     ruby_tag  = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><rt>{chu_im}</rt><rp>)</rp></ruby>'
        chu_im_sheet.range('A' + str(i)).value = ruby_tag
        i += 1
        row += 1
        continue
    
    #=========================================================
    # 備妥注音時需參考用的資料
    #=========================================================
    siann_index = int(source_sheet.range('F' + str(row)).value) - 1
    un_index = int(source_sheet.range('G' + str(row)).value) - 1
    tiau_ho =  int(source_sheet.range('E' + str(row)).value)

    #=========================================================
    # 使用注音碼，取得各式之〖 注音／拼音〗
    #=========================================================
    if zhu_im_method == 'sip_ngoo_im':
        # 輸出十五音
        chu_im = ji.get_sip_ngoo_im_chu_im(siann_index, un_index, tiau_ho)
    elif zhu_im_method == 'TPS':
        # 方音符號注音
        chu_im = ji.get_TPS_chu_im(siann_index, un_index, tiau_ho)
    elif zhu_im_method == 'POJ':
        # 輸出白話字拼音
        chu_im = ji.get_POJ_chu_im(siann_index, un_index, tiau_ho)
    elif zhu_im_method == 'TL':
        # 輸出羅馬拼音
        chu_im = ji.get_TL_chu_im(siann_index, un_index, tiau_ho)
    elif zhu_im_method == 'BP':
        chu_im  = ji.get_BP_chu_im(siann_index, un_index, tiau_ho)
        # 輸出閩拼拼音
        # BP_chu_im1 = ji.get_BP_chu_im(siann_index, un_index, tiau_ho)

    #=========================================================
    # 將已注音之漢字加入【漢字注音表】
    #=========================================================
    # if zhu_im_method == 'sip_ngoo_im':
    #     ruby_tag = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><rtc>{chu_im}</rtc><rp>)</rp></ruby>'
    # else:
    #     ruby_tag  = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><rt>{chu_im}</rt><rp>)</rp></ruby>'
    ruby_tag = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><{rt_tag}>{chu_im}</{rt_tag}><rp>)</rp></ruby>'
    chu_im_sheet.range('A' + str(i)).value = ruby_tag
    
    #=========================================================
    # 調整讀取來源；寫入標的各工作表
    #=========================================================
    i += 1
    row += 1

#=========================================================
# 輸出 </div> 
#=========================================================
html_str = "</p></div>"
# html_str = """
#   </p>
# </div>
# """
chu_im_sheet.range('A' + str(i)).value = html_str